Load Libraries

In [23]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.extra.rate_limiter import RateLimiter

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium --yes
import folium # map rendering library

import math

Load information about national capitals from Wiki:

In [2]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_national_capitals_by_population',header=0)[1]

And see first 5:

In [3]:
df.head()

,Rank,Country/Territory,Capital,Population,Year,% ofcountry'spopulation
0,1,China PR,Beijing,"21,542,000[1]",2010,1.5%
1,2,Japan,Tokyo,"13,929,286[2]",2017,11.03%
2,3,Russia,Moscow,"12,506,468[3]",2011,8.52%
3,4,DR Congo,Kinshasa,"11,855,000[4]",2012,12.9%
4,5,Indonesia,Jakarta,"10,075,310[5]",2011,3.76%


In some rows we can see two cities or city name + something. We can't take coordinates with wrong name city, so will check and correct it. In 155 row we have two "Populations", so correct it too:

In [4]:
chg=[(57,"Guayaquil"),(81,"Guatemala"),(89,"Naypyidaw"),(100,"Jerusalem"),
     (154,"El Aaiún"),(163,"Berne"),(171,"Nicosia"),(200,"Ramallah"),(231,"Yaren"),(235,"Vatican City"),(239,"Brades")]

In [5]:
for i in range(len(chg)):
    df.loc[chg[i][0],"Capital"]=chg[i][1]
df.loc[154,"Population"]="194668"

In [6]:
df.head()

,Rank,Country/Territory,Capital,Population,Year,% ofcountry'spopulation
0,1,China PR,Beijing,"21,542,000[1]",2010,1.5%
1,2,Japan,Tokyo,"13,929,286[2]",2017,11.03%
2,3,Russia,Moscow,"12,506,468[3]",2011,8.52%
3,4,DR Congo,Kinshasa,"11,855,000[4]",2012,12.9%
4,5,Indonesia,Jakarta,"10,075,310[5]",2011,3.76%


Creating two DataFrame with capital's coordinates and if we take Error in taking coordinate:

In [7]:
coord=pd.DataFrame(columns=["Capital","Latitude","Longitude"])
err=pd.DataFrame(columns=["Number","Name"])

And take coordinate from Nomutatim. When we take coordinate, we print result for viewing progress:

In [8]:
for i in df.index:
    geolocator = Nominatim(user_agent="tr_explorer")
    address =df.loc[i,"Capital"]    
    try:
        location = geolocator.geocode(address,timeout=1)
        latitude = location.latitude
        longitude = location.longitude        
    except:
        coord=coord.append({"Capital":address,"Latitude":np.nan,"Longitude":np.nan},ignore_index=True)
        err=err.append({"Number":i,"Name":address},ignore_index=True)
        print('The geograpical coordinate of {} denied'.format(address))
    else:
        coord=coord.append({"Capital":address,"Latitude":latitude,"Longitude":longitude},ignore_index=True)
        print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Beijing are 39.906217, 116.3912757.
The geograpical coordinate of Tokyo are 35.6828387, 139.7594549.
The geograpical coordinate of Moscow are 55.4792046, 37.3273304.
The geograpical coordinate of Kinshasa are -4.3217055, 15.3125974.
The geograpical coordinate of Jakarta are -6.1753942, 106.827183.
The geograpical coordinate of Seoul are 37.564982549999996, 126.93921080358436.
The geograpical coordinate of Cairo are 30.048819, 31.243666.
The geograpical coordinate of Mexico City are 19.4326296, -99.1331785.
The geograpical coordinate of Dhaka are 23.7593572, 90.3788136.
The geograpical coordinate of London are 51.5073219, -0.1276474.
The geograpical coordinate of Lima are -12.0621065, -77.0365256.
The geograpical coordinate of Tehran are 35.7006177, 51.4013785.
The geograpical coordinate of Bangkok are 13.7542529, 100.493087.
The geograpical coordinate of Hanoi are 21.0294498, 105.8544441.
The geograpical coordinate of Riyadh are 24.6319692, 46.7150648.
The

The geograpical coordinate of Macau are 22.1757605, 113.5514142.
The geograpical coordinate of Edinburgh are 55.9533456, -3.1883749.
The geograpical coordinate of Djibouti (city) are 11.8145966, 42.8453061.
The geograpical coordinate of Yamoussoukro are 6.809107, -5.273263.
The geograpical coordinate of Bissau are 11.861324, -15.583055.
The geograpical coordinate of Tallinn are 59.4372155, 24.7453688.
The geograpical coordinate of Bratislava are 48.1516988, 17.1093063.
The geograpical coordinate of San Juan denied
The geograpical coordinate of Canberra are -35.2975906, 149.1012676.
The geograpical coordinate of Wellington are -41.2887953, 174.7772114.
The geograpical coordinate of Sarajevo are 43.8519774, 18.3866868.
The geograpical coordinate of Juba are 4.8472017, 31.5951655.
The geograpical coordinate of Cardiff are 51.4816546, -3.1791934.
The geograpical coordinate of Belfast are 54.5964411, -5.9302761.
The geograpical coordinate of San José are 37.3361905, -121.8905833.
The geogra

Will see Errors:

In [9]:
err

,Number,Name
0,56,Beirut
1,65,Bamako
2,74,Mogadishu
3,83,Ouagadougou
4,93,Bishkek
5,103,Tirana
6,112,Asmara
7,122,Libreville
8,133,San Juan
9,142,Vientiane


If we saw error with name of Capital, we add it in our list "chg" upper. Try again take cooridinates, and if we success delete row from DataFrame err:

In [10]:
for i in err.index:
    geolocator = Nominatim(user_agent="tr_explorer")
    address =err.loc[i,"Name"]
    try:
        location = geolocator.geocode(address,timeout=1)
        latitude = location.latitude
        longitude = location.longitude        
    except:
        print('The geograpical coordinate of {} denied'.format(address))
    else:
        coord.loc[err.loc[i,"Number"],"Latitude"]=latitude
        coord.loc[err.loc[i,"Number"],"Longitude"]=longitude
        err=err.drop(i,axis=0)
        print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Beirut are 33.8959203, 35.47843.
The geograpical coordinate of Bamako are 12.60503275, -7.986513673439357.
The geograpical coordinate of Mogadishu are 2.042778, 45.338564.
The geograpical coordinate of Ouagadougou are 12.368148, -1.527085.
The geograpical coordinate of Bishkek are 42.8765615, 74.6070079.
The geograpical coordinate of Tirana are 41.31588575, 19.900912164461147.
The geograpical coordinate of Asmara are 15.3389667, 38.9326763.
The geograpical coordinate of Libreville are 0.390002, 9.454001.
The geograpical coordinate of San Juan are 18.465299, -66.116666.
The geograpical coordinate of Vientiane are 17.9640988, 102.6133707.
The geograpical coordinate of Porto-Novo are 6.4990718, 2.6253361.
The geograpical coordinate of Georgetown are 6.8025766, -58.1628612.
The geograpical coordinate of Nicosia are 35.1739302, 33.364726.
The geograpical coordinate of São Tomé are 0.3389242, 6.7313031.
The geograpical coordinate of Banjul are 13.441346450000001

Look Error again:

In [11]:
err

,Number,Name


It's empty now. If we take error again we try and try to all succesful. Look to first 5 rows in DataFrame coord:

In [12]:
coord.head()

,Capital,Latitude,Longitude
0,Beijing,39.906217,116.391276
1,Tokyo,35.682839,139.759455
2,Moscow,55.479205,37.327330
3,Kinshasa,-4.321706,15.312597
4,Jakarta,-6.175394,106.827183


Add columns Latitude and Longitude in our main DataFrame and see result

In [13]:
df["Latitude"]=coord["Latitude"]
df["Longitude"]=coord["Longitude"]

In [14]:
df.head()

,Rank,Country/Territory,Capital,Population,Year,% ofcountry'spopulation,Latitude,Longitude
0,1,China PR,Beijing,"21,542,000[1]",2010,1.5%,39.906217,116.391276
1,2,Japan,Tokyo,"13,929,286[2]",2017,11.03%,35.682839,139.759455
2,3,Russia,Moscow,"12,506,468[3]",2011,8.52%,55.479205,37.327330
3,4,DR Congo,Kinshasa,"11,855,000[4]",2012,12.9%,-4.321706,15.312597
4,5,Indonesia,Jakarta,"10,075,310[5]",2011,3.76%,-6.175394,106.827183


Check lists for Errors:

In [15]:
for i in coord.index:
    if coord.loc[i,"Capital"]!=df.loc[i,"Capital"]:
        print("Error in {} row".format(i))

Check types in our DataFrame:

In [16]:
df.dtypes

Rank                        object
Country/Territory           object
Capital                     object
Population                  object
Year                        object
% ofcountry'spopulation     object
Latitude                   float64
Longitude                  float64
dtype: object

Correct Population to integer and drop column "Rank"

In [17]:
for i in df.index:
    b=df.loc[i,"Population"].find("[")
    if b!=-1:
        df.loc[i,"Population"]=df.loc[i,"Population"][:b]
    b=df.loc[i,"Population"].find("(")
    if b!=-1:
        df.loc[i,"Population"]=df.loc[i,"Population"][:b-1]
    df.loc[i,"Population"]=df.loc[i,"Population"].replace(",","")
        

In [18]:
df[["Population"]]=df[["Population"]].astype("int")

In [19]:
df=df.drop("Rank",axis=1)

See result:

In [20]:
df.head()

,Country/Territory,Capital,Population,Year,% ofcountry'spopulation,Latitude,Longitude
0,China PR,Beijing,21542000,2010,1.5%,39.906217,116.391276
1,Japan,Tokyo,13929286,2017,11.03%,35.682839,139.759455
2,Russia,Moscow,12506468,2011,8.52%,55.479205,37.327330
3,DR Congo,Kinshasa,11855000,2012,12.9%,-4.321706,15.312597
4,Indonesia,Jakarta,10075310,2011,3.76%,-6.175394,106.827183


Because every Capital have different size and we have only 99,500 Regular Calls/Day, we take some count of venue for every city based by Population:

In [21]:
df["CountVenue"]=df["Population"]//100000*5

In [24]:
for i in df.index:
    if df.loc[i,"Population"]<2000000:
        df.loc[i,"CountVenue"]=100
    else:
        df.loc[i,"CountVenue"]=math.ceil(df.loc[i,"CountVenue"]/100)*100

In [25]:
df.head(10)

,Country/Territory,Capital,Population,Year,% ofcountry'spopulation,Latitude,Longitude,CountVenue
0,China PR,Beijing,21542000,2010,1.5%,39.906217,116.391276,1100
1,Japan,Tokyo,13929286,2017,11.03%,35.682839,139.759455,700
2,Russia,Moscow,12506468,2011,8.52%,55.479205,37.327330,700
3,DR Congo,Kinshasa,11855000,2012,12.9%,-4.321706,15.312597,600
4,Indonesia,Jakarta,10075310,2011,3.76%,-6.175394,106.827183,500
5,South Korea,Seoul,9838892,2015,19.03%,37.564983,126.939211,500
6,Egypt,Cairo,9500000,2012,9.54%,30.048819,31.243666,500
7,Mexico,Mexico City,8918653,2015,7.05%,19.432630,-99.133178,500
8,Bangladesh,Dhaka,8906039,2011,5.52%,23.759357,90.378814,500
9,United Kingdom England,London,8908081,2015,13.19%,51.507322,-0.127647,500


Checked sum of Count:

In [26]:
df["CountVenue"].sum()

35000

Point all marks on the world map and look at this:

In [27]:
#Create world map
world_map=folium.Map()

# add markers to map
for lat, lng, country, capital in zip(df['Latitude'], df['Longitude'], df['Country/Territory'], df['Capital']):
    label = '{}, {}'.format(capital, country)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(world_map)

world_map

As we can see some of island have wrong coordinates, and we point it on the continantes as "Victoria, Seychelles" on the Australia. Create list with mistakes, where we put number of row and city's name with island's name:

In [28]:
chg=[(193,"Victoria, Seychelles"),(18,"Santiago, Chile"),(140,"San José, Costa Rica"),
     (230,"The Valley, Anguilla"),(209,"Roseau, Dominica"),(198,"Isle of Man"),
    (236,"Jamestown, Saint Helena"),(203,"St. John's, Antigua and Barbuda"),
     (197,"George Town, Cayman Islands"),(179,"Dili, East Timor"),(126,"Macau, Macau"),
    (233,"Hamilton, Bermuda"),(219,"Saint-Pierre, Saint Pierre and Miquelon")]

In [29]:
for i in range(len(chg)):
    geolocator = Nominatim(user_agent="tr_explorer")
    address =chg[i][1]
    try:
        location = geolocator.geocode(address,timeout=1)
        latitude = location.latitude
        longitude = location.longitude        
    except:
        print('The geograpical coordinate of {} denied'.format(address))
    else:
        df.loc[chg[i][0],"Latitude"]=latitude
        df.loc[chg[i][0],"Longitude"]=longitude        
        print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Victoria, Seychelles are -4.6232085, 55.452359.
The geograpical coordinate of Santiago, Chile are -33.4377968, -70.6504451.
The geograpical coordinate of San José, Costa Rica are 9.9325427, -84.0795782.
The geograpical coordinate of The Valley, Anguilla are 18.2145861, -63.0517759.
The geograpical coordinate of Roseau, Dominica are 15.2991923, -61.3872868.
The geograpical coordinate of Isle of Man are 54.2358167, -4.514598745698255.
The geograpical coordinate of Jamestown, Saint Helena are -15.9277296, -5.7160872.
The geograpical coordinate of St. John's, Antigua and Barbuda are 17.1184569, -61.8448509.
The geograpical coordinate of George Town, Cayman Islands are 19.2953549, -81.3807776.
The geograpical coordinate of Dili, East Timor are -8.5536809, 125.5784093.
The geograpical coordinate of Macau, Macau are -5.1133665, -36.6349957.
The geograpical coordinate of Hamilton, Bermuda are 32.2956076, -64.7827048.
The geograpical coordinate of Saint-Pierre, Sai

We can't take correct Brasilia's coordinates so we take it from Wiki. Atafu have 0 venues so we drop it: 

In [30]:
#Brasilia, Brazilia
df.iloc[34,5]=-15.793889
df.iloc[34,6]=-47.882778
#Atafu, Tokelau (NZ)
df.drop(238,axis=0,inplace=True)

Recreate world map:

In [31]:
#Create world map
world_map=folium.Map()

# add markers to map
for lat, lng, country, capital in zip(df['Latitude'], df['Longitude'], df['Country/Territory'], df['Capital']):
    label = '{}, {}'.format(capital, country)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(world_map)

world_map

Define Foursquare Credentials and Version. For save secrets we do it from file:

In [32]:
with open("Foursquare.txt","r") as file1:
    CLIENT_ID = file1.readline()
    CLIENT_ID = CLIENT_ID[:(-1)] #delete "/n" from readline
    CLIENT_SECRET = file1.readline()
VERSION = "20200101"
LIMIT=100

Because we can take only 100 venues from 1 call, calcucate how big radius we need for this. For this create function to get top from coordinates with some radius:

In [33]:
def getCountVenues(name,lat,lng,radius):
    venues_list=[]
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
            
    # make the GET request        
    results = requests.get(url).json()["response"]['groups'][0]['items']

    # return only relevant information for each nearby venue
    if len(results)!=0:
        venues_list.append([(
           name, 
           lat, 
           lng, 
           v['venue']['name'], 
           v['venue']['location']['lat'], 
           v['venue']['location']['lng'],  
           v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                      'Neighborhood Latitude', 
                      'Neighborhood Longitude', 
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']
        return len(nearby_venues.index)
    else:
        return 0

Maximum radius from Foursquare is 100.000 meters, so we push this value:

In [34]:
df["RadiusRequest"]=100000

In [35]:
df.head()

,Country/Territory,Capital,Population,Year,% ofcountry'spopulation,Latitude,Longitude,CountVenue,RadiusRequest
0,China PR,Beijing,21542000,2010,1.5%,39.906217,116.391276,1100,100000
1,Japan,Tokyo,13929286,2017,11.03%,35.682839,139.759455,700,100000
2,Russia,Moscow,12506468,2011,8.52%,55.479205,37.327330,700,100000
3,DR Congo,Kinshasa,11855000,2012,12.9%,-4.321706,15.312597,600,100000
4,Indonesia,Jakarta,10075310,2011,3.76%,-6.175394,106.827183,500,100000


And calculate radius for every capital:

In [36]:
for i in df.index:    
    r=df.loc[i,"RadiusRequest"]
    if r==100000:                        #if r = 100 000 then we don't change it
        l=getCountVenues(df.iloc[i,1],df.iloc[i,5],df.iloc[i,6],r)
        if l==100:                       # if l < 100 then radius for this capital is 100 000
            r=500                        # go from bottom to our radius
            l=getCountVenues(df.iloc[i,1],df.iloc[i,5],df.iloc[i,6],r)
            while l!=100:
                r=r*2
                l=getCountVenues(df.iloc[i,1],df.iloc[i,5],df.iloc[i,6],r)           
        df.loc[i,"RadiusRequest"]=r
        if r==100000:
            df.loc[i,"CountVenue"]=l
    if r>100000:          # if r > 100 000 then for this town we have venue's counts close to 100
        df.loc[i,"CountVenue"]=100
    print("For {} need radius {}.".format(df.iloc[i,1],r)) 

For Beijing need radius 2000.
For Tokyo need radius 500.
For Moscow need radius 16000.
For Kinshasa need radius 100000.
For Jakarta need radius 2000.
For Seoul need radius 1000.
For Cairo need radius 1000.
For Mexico City need radius 500.
For Dhaka need radius 100000.
For London need radius 500.
For Lima need radius 2000.
For Tehran need radius 2000.
For Bangkok need radius 1000.
For Hanoi need radius 1000.
For Riyadh need radius 2000.
For Hong Kong need radius 1000.
For Bogotá need radius 1000.
For Baghdad need radius 100000.
For Santiago need radius 1000.
For Singapore need radius 2000.
For Ankara need radius 16000.
For Berlin need radius 1000.
For Damascus need radius 128000.
For Algiers need radius 100000.
For Madrid need radius 500.
For Pyongyang need radius 100000.
For Kabul need radius 100000.
For Nairobi need radius 2000.
For Addis Ababa need radius 100000.
For Buenos Aires need radius 1000.
For Rome need radius 1000.
For Kiev need radius 500.
For Yaoundé need radius 100000.
Fo

IndexError: single positional indexer is out-of-bounds

We can take only 100 venues from 1 call, so we need more points for 1 city. Calculate degree/km, because we don't need high accurance, we can divide 360 degree for equator length:

In [38]:
dl=360/40076
dl

0.008982932428386067

And create more points around our capital's coordinates:

In [39]:
points=pd.DataFrame(columns=["Capital","Latitude","Longitude","RadiusRequest"])
for i in df.index:
    c=df.loc[i,"CountVenue"]
    if c<=100:
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]},ignore_index=True)
    elif c==200:
        l=dl*df.loc[i,"RadiusRequest"]/1000
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]-l},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]+l},ignore_index=True)
    elif c==300:
        l=2*dl*df.loc[i,"RadiusRequest"]/1000
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]},ignore_index=True)                              
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]-l},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]+l},ignore_index=True)
    elif c==400:
        l=1.4*dl*df.loc[i,"RadiusRequest"]/1000
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]-l},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]+l},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"]-l,
                               "Longitude":df.loc[i,"Longitude"]},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"]+l,
                               "Longitude":df.loc[i,"Longitude"]},ignore_index=True)
    elif c==500:
        l=2*dl*df.loc[i,"RadiusRequest"]/1000
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]},ignore_index=True)                              
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]-l},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]+l},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"]-l,
                               "Longitude":df.loc[i,"Longitude"]},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"]+l,
                               "Longitude":df.loc[i,"Longitude"]},ignore_index=True)
    elif c==700:
        l=2*dl*df.loc[i,"RadiusRequest"]/1000
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]},ignore_index=True)                              
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]-l},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]+l},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"]-l,
                               "Longitude":df.loc[i,"Longitude"]},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"]+l,
                               "Longitude":df.loc[i,"Longitude"]},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"]-l,
                               "Longitude":df.loc[i,"Longitude"]-l},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"]+l,
                               "Longitude":df.loc[i,"Longitude"]+l},ignore_index=True)
    elif c==1100:
        l=2*dl*df.loc[i,"RadiusRequest"]/1000
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]},ignore_index=True)                              
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]-l},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"],
                               "Longitude":df.loc[i,"Longitude"]+l},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"]-l,
                               "Longitude":df.loc[i,"Longitude"]},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"]+l,
                               "Longitude":df.loc[i,"Longitude"]},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"]-l,
                               "Longitude":df.loc[i,"Longitude"]-l},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"]+l,
                               "Longitude":df.loc[i,"Longitude"]+l},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"]-l,
                               "Longitude":df.loc[i,"Longitude"]+l},ignore_index=True)
        points=points.append({"Capital":df.loc[i,"Capital"],
                               "RadiusRequest":df.loc[i,"RadiusRequest"],
                               "Latitude":df.loc[i,"Latitude"]+l,
                               "Longitude":df.loc[i,"Longitude"]-l},ignore_index=True)
                              

In [40]:
points.head(10)

,Capital,Latitude,Longitude,RadiusRequest
0,Beijing,39.906217,116.391276,2000
1,Beijing,39.906217,116.355344,2000
2,Beijing,39.906217,116.427207,2000
3,Beijing,39.870285,116.391276,2000
4,Beijing,39.942149,116.391276,2000
5,Beijing,39.870285,116.355344,2000
6,Beijing,39.942149,116.427207,2000
7,Beijing,39.870285,116.427207,2000
8,Beijing,39.942149,116.355344,2000
9,Tokyo,35.682839,139.759455,500


See how many venues we can take with this points, and will be sure that we don't have any 0 venue's count:

In [41]:
points["Count"]=0
for i in points.index:
    points.loc[i,"Count"]=getCountVenues(points.iloc[i,0],points.iloc[i,1],
                                         points.iloc[i,2],points.iloc[i,3])
    print("{},{} venue's count= {}".format(i,points.iloc[i,0],points.loc[i,"Count"]))

0,Beijing venue's count= 100
1,Beijing venue's count= 69
2,Beijing venue's count= 100
3,Beijing venue's count= 46
4,Beijing venue's count= 100
5,Beijing venue's count= 34
6,Beijing venue's count= 100
7,Beijing venue's count= 41
8,Beijing venue's count= 64
9,Tokyo venue's count= 100
10,Tokyo venue's count= 11
11,Tokyo venue's count= 100
12,Tokyo venue's count= 100
13,Tokyo venue's count= 100
14,Tokyo venue's count= 58
15,Tokyo venue's count= 100
16,Moscow venue's count= 100
17,Moscow venue's count= 100
18,Moscow venue's count= 100
19,Moscow venue's count= 52
20,Moscow venue's count= 100
21,Moscow venue's count= 20
22,Moscow venue's count= 100
23,Kinshasa venue's count= 34
24,Jakarta venue's count= 100
25,Jakarta venue's count= 100
26,Jakarta venue's count= 100
27,Jakarta venue's count= 100
28,Jakarta venue's count= 100
29,Seoul venue's count= 100
30,Seoul venue's count= 100
31,Seoul venue's count= 49
32,Seoul venue's count= 100
33,Seoul venue's count= 36
34,Cairo venue's count= 100
35,C

270,Kingstown venue's count= 91
271,Apia venue's count= 13
272,Port Vila venue's count= 11
273,Monaco venue's count= 100
274,Banjul venue's count= 31
275,Tarawa venue's count= 9
276,Oranjestad venue's count= 100
277,Victoria venue's count= 95
278,Gibraltar venue's count= 100
279,Saint Helier venue's count= 100
280,Bandar Seri Begawan venue's count= 100
281,George Town venue's count= 96
282,Douglas venue's count= 100
283,Papeete venue's count= 59
284,Ramallah venue's count= 100
285,Majuro venue's count= 4
286,Andorra la Vella venue's count= 100
287,St. John's venue's count= 65
288,Nukuʻalofa venue's count= 8
289,Tórshavn venue's count= 42
290,St. Peter Port venue's count= 100
291,Belmopan venue's count= 70
292,Nuuk (Godthåb) venue's count= 5
293,Roseau venue's count= 100
294,Basseterre venue's count= 100
295,Mariehamn venue's count= 74
296,Charlotte Amalie venue's count= 100
297,Palikir venue's count= 4
298,Road Town venue's count= 100
299,St. George's venue's count= 100
300,Valletta ve

In [48]:
points.to_excel("temp.xlsx",index=False)
df.to_excel("capitals_2.xlsx",index=False)